In [1]:
!pip install pandas
!pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
df = pd.read_csv('/content/questions.csv')
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404346,404346,789792,789793,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404347,404347,789794,789795,Do you believe there is life after death?,Is it true that there is life after death?,1
404348,404348,789796,789797,What is one coin?,What's this coin?,0
404349,404349,789798,789799,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [4]:
df = df.dropna(subset=['question1', 'question2'])
df = df[df['question1'].str.len() > 0]
df = df[df['question2'].str.len() > 0]
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404346,404346,789792,789793,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404347,404347,789794,789795,Do you believe there is life after death?,Is it true that there is life after death?,1
404348,404348,789796,789797,What is one coin?,What's this coin?,0
404349,404349,789798,789799,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [5]:

unique_questions = {}
next_id = 1

for question in df['question1']:
  if question not in unique_questions:
    unique_questions[question] = next_id
    next_id += 1

for question in df['question2']:
  if question not in unique_questions:
    unique_questions[question] = next_id
    next_id += 1


In [6]:
question_id_to_question={}
for question, question_id in unique_questions.items():
  question_id_to_question[question_id]=question

In [7]:

df['q1_unique_id'] = df['question1'].map(unique_questions)
df['q2_unique_id'] = df['question2'].map(unique_questions)
df


,id,qid1,qid2,question1,question2,is_duplicate,q1_unique_id,q2_unique_id
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,1,290472
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,2,16742
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,3,290473
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,4,290474
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,5,290475
...,...,...,...,...,...,...,...,...
404346,404346,789792,789793,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0,232861,457520
404347,404347,789794,789795,Do you believe there is life after death?,Is it true that there is life after death?,1,9523,81463
404348,404348,789796,789797,What is one coin?,What's this coin?,0,290469,537384
404349,404349,789798,789799,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0,290470,537385


In [8]:

print(len(unique_questions))


537386


In [9]:

df.to_csv('modified_questions.csv', index=False)


In [10]:

import networkx as nx


graph = nx.Graph()

# Add nodes for each unique question
for question, question_id in unique_questions.items():
    graph.add_node(question_id)

In [11]:

# Add edges with weight 1 for each node to itself
for question_id in unique_questions.values():
    graph.add_edge(question_id, question_id, weight=1)


In [12]:

# Iterate through the DataFrame
for index, row in df.iterrows():
    if row['is_duplicate']:
        q1_id = row['q1_unique_id']
        q2_id = row['q2_unique_id']

        # Add edges between q1_id and q2_id with weight 1
        graph.add_edge(q1_id, q2_id, weight=1)
        graph.add_edge(q2_id, q1_id, weight=1)


In [13]:

# Assuming df is defined from the previous code

df['k'] = 0
df

,id,qid1,qid2,question1,question2,is_duplicate,q1_unique_id,q2_unique_id,k
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,1,290472,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,2,16742,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,3,290473,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,4,290474,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,5,290475,0
...,...,...,...,...,...,...,...,...,...
404346,404346,789792,789793,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0,232861,457520,0
404347,404347,789794,789795,Do you believe there is life after death?,Is it true that there is life after death?,1,9523,81463,0
404348,404348,789796,789797,What is one coin?,What's this coin?,0,290469,537384,0
404349,404349,789798,789799,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0,290470,537385,0


In [14]:

from collections import defaultdict
# Assuming unique_questions and df are defined from the previous code
# and the graph is built as in the previous response.
transitive_positive_dependence_k=defaultdict(lambda: defaultdict(int))

def bfs_with_distances(graph, start_node):
  distances = {}  # Initialize an empty dictionary for distances
  distances[start_node] = 0  # Set distance from start node to itself as 0
  queue = [(start_node, 0)]  # Initialize the queue with start node and distance 0
  visited = {start_node}  # Initialize visited set with the start node

  while queue:
    current_node, current_distance = queue.pop(0)  # Get the next node from the queue
    for neighbor in graph.neighbors(current_node):  # Iterate through neighbors
      if neighbor not in visited:  # If neighbor not visited yet
        visited.add(neighbor)  # Mark neighbor as visited
        distances[neighbor] = current_distance + 1  # Set distance for neighbor
        queue.append((neighbor, current_distance + 1))  # Add neighbor to queue

  return distances


for i in range(1,  len(unique_questions) + 1): # Iterate up to 100 or the number of unique questions
    if i in graph.nodes():
        distances_from_start = bfs_with_distances(graph, i)
        # Process the distances (e.g., print or store them)
        for node, distance in distances_from_start.items():
            transitive_positive_dependence_k[i][node] = distance
    else:
        print(f"Node {i} not found in the graph.")
transitive_positive_dependence_k[6]

defaultdict(int, {6: 0, 290476: 1})

In [15]:
import pandas as pd

# Assuming df, transitive_positive_dependence_k, and unique_questions are defined

new_rows = []  # List to store new rows

for q1_id in transitive_positive_dependence_k:
    for q2_id, distance in transitive_positive_dependence_k[q1_id].items():
        if distance > 0 and q1_id != q2_id:  # Consider distances > 0 and different questions
            new_rows.append({'qid1': None, 'qid2': None, 'question1': question_id_to_question[q1_id], 'question2': question_id_to_question[q2_id], 'is_duplicate': 1, 'q1_unique_id': q1_id, 'q2_unique_id': q2_id, 'k': distance})

# Create a DataFrame from the new rows
new_df = pd.DataFrame(new_rows)

# Concatenate the new DataFrame with the original DataFrame
df = pd.concat([df, new_df], ignore_index=True)
df

,id,qid1,qid2,question1,question2,is_duplicate,q1_unique_id,q2_unique_id,k
0,0.0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,1,290472,0
1,1.0,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,2,16742,0
2,2.0,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,3,290473,0
3,3.0,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,4,290474,0
4,4.0,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,5,290475,0
...,...,...,...,...,...,...,...,...,...
863679,NaN,None,None,How do I become a ethical hacker?,What are requirements to become a ethical hacker?,1,537357,144418,1
863680,NaN,None,None,How do Russian politics and geostrategy affect...,How does Russian politics affect Australia and...,1,537359,290436,1
863681,NaN,None,None,What would happen if I put milk instead of wat...,What happens if you put milk in a coffee maker?,1,537377,290462,1
863682,NaN,None,None,What are some outfit ideas wear to a frat them...,What are some outfit ideas to wear to a frat p...,1,537381,290466,1


In [16]:

df.to_csv('final_modified_questions.csv', index=False)

In [17]:

print(len(df))

863684


In [18]:


# Assuming df, transitive_positive_dependence_k, and unique_questions are defined from the previous code
# and the graph is already created.
for q1_id in transitive_positive_dependence_k:
    for q2_id, distance in transitive_positive_dependence_k[q1_id].items():
        if distance > 0 and q1_id != q2_id:
            if not graph.has_edge(q1_id, q2_id):
                graph.add_edge(q1_id, q2_id, weight=1)


In [19]:


# Assuming df and graph are defined from the previous code

for index, row in df.iterrows():
    if not row['is_duplicate']:
        q1_id = row['q1_unique_id']
        q2_id = row['q2_unique_id']

        # Add edges between q1_id and q2_id with weight 0
        graph.add_edge(q1_id, q2_id, weight=0)
        graph.add_edge(q2_id, q1_id, weight=0)


In [20]:

clique_of_question = [0] *( len(unique_questions)+1)

In [21]:
next_clique=1
cliques={}
for q_id in transitive_positive_dependence_k:
  if clique_of_question[q_id] ==0:
    clique_of_question[q_id]=next_clique
    cliques[next_clique]=[q_id]
    next_clique+=1
    for each in transitive_positive_dependence_k[q_id]:
      clique_of_question[each]=clique_of_question[q_id]
      if each not in cliques[clique_of_question[q_id]]:  # Check for duplicates before adding
                cliques[clique_of_question[q_id]].append(each)
next_clique

448183

In [22]:

clique_sizes = [len(clique) for clique in cliques.values()]
sum_of_clique_sizes = sum(clique_sizes)
print(f"The sum of the sizes of all cliques is: {sum_of_clique_sizes}")


The sum of the sizes of all cliques is: 537386


In [23]:
import pandas as pd

new_rows = []  # List to store new rows

for index, row in df.iterrows():
    if not row['is_duplicate']:
        q1_id = row['q1_unique_id']
        q2_id = row['q2_unique_id']
        clique1 = cliques[clique_of_question[q1_id]]
        clique2 = cliques[clique_of_question[q2_id]]

        # Add edges and create new rows for new edges
        for node1 in clique1:
            for node2 in clique2:
                if node1 != node2 and not graph.has_edge(node1, node2):  # Avoid self-loops and existing edges
                    graph.add_edge(node1, node2, weight=0)
                    graph.add_edge(node2, node1, weight=0)
                    new_rows.append({'qid1': None, 'qid2': None, 'question1': question_id_to_question[node1], 'question2': question_id_to_question[node2], 'is_duplicate': 0, 'q1_unique_id': node1, 'q2_unique_id': node2, 'k': 1})

# Create a DataFrame from the new rows
new_df = pd.DataFrame(new_rows)

# Concatenate the new DataFrame with the original DataFrame
df = pd.concat([df, new_df], ignore_index=True)

In [24]:
new_df

,qid1,qid2,question1,question2,is_duplicate,q1_unique_id,q2_unique_id,k
0,None,None,Which are the best digital marketing firms in ...,Which is the best digital marketing institute ...,0,36542,290487,1
1,None,None,What are the questions should not ask on Quora?,What's the best question to ask in Quora?,0,23,69907,1
2,None,None,What are the questions should not ask on Quora?,What are good questions to ask on Quora?,0,23,344049,1
3,None,None,What are the questions should not ask on Quora?,Why should I ask Quora?,0,23,143946,1
4,None,None,What are some questions that shouldn't be aske...,What's the best question to ask in Quora?,0,455083,69907,1
...,...,...,...,...,...,...,...,...
297510,None,None,Which is the best phone at 12000?,Which mobile is better under 15k?,0,185463,305326,1
297511,None,None,Which is the best phone at 12000?,What are some good smartphones under 15k?,0,185463,66382,1
297512,None,None,Which is the best phone at 12000?,Which is best phone? Around 15 k?,0,185463,221380,1
297513,None,None,How can we prove causal relationship between b...,Why does CO2 contribute more to global warming...,0,200795,537373,1


In [25]:
len(df)

1161199